# David Raveh and Rafael I. Nepomechie

## david.raveh@rutgers.edu               

## nepomechie@miami.edu

# The purpose of this code is to prepare the Bethe states $|B^L_M(\vec k)\rangle$, based on https://arxiv.org/abs/2403.03283

In [1]:
import qiskit 
qiskit.version.get_version_info()

'1.0.2'

In [2]:
from qiskit import *
import numpy as np
import itertools
from qiskit import QuantumRegister
from qiskit import QuantumCircuit
from qiskit.circuit.library import RYGate
from qiskit.circuit.library import UGate
np.set_printoptions(suppress=True)
np.set_printoptions(threshold=np.inf)

## The function $f(w,\vec k)$ for the *closed* XXZ chain

The Hamiltonian for the closed spin-1/2 XXZ chain, which depends on one parameter $\Delta$, is given by

$$ \mathcal{H} = \tfrac{1}{4}\sum_{n=1}^L 
    \left(\sigma^x_n \sigma^x_{n+1} + \sigma^y_n \sigma^y_{n+1} + \Delta\, \sigma^z_n \sigma^z_{n+1} \right) \,, \qquad \vec{\sigma}_{L+1} = \vec{\sigma}_1 $$

Here we make the closed-chain function $f(w,\vec k)$ for a given set of Bethe roots $\vec k=[k_1,\dots,k_M]$

$$f(w, \vec{k}) = \sum_{\sigma\in S_M} \varepsilon(\sigma)\, A(k_{\sigma(1)}, \cdots, k_{\sigma(M)})\, 
    e^{i \sum_{j=1}^M k_{\sigma(j)} x_j} $$

where $A(k_1, \ldots, k_M)$ is defined in Appendix A.

In [3]:
#mylist is a collection of permutations t of the numbers 0 to M-1
def mylist(M): 
    return list(itertools.permutations(range(M)))

# inversion number of t=(0,1,2,...,M-1)
def inv(t):
    x=0
    for j in range(M):
        for i in range(j):
            if t[i]>t[j]:
                x=x+1
    return x

# sign of permutation t
def e(t):
    return (-1)**inv(t)
    
def s(k,kp):
    return 1-2*Delta*np.exp(1.j*kp)+np.exp(1.j*(k+kp))

# k=[k0,...,kM-1]
def Aclosed(k):
    x=1
    for l in range(len(k)):
        for j in range(l):
            x=x*s(k[l],k[j])
    return x

# permutes the bethe roots k into an order determined by t: [k0,k1,k2] -> [k1,k2,k0]
def kperm(k,t):
    y=[]
    for i in t:
        y.append(k[i])
    return y

# positions of the ones in w
def positions(w):
    x=[]
    for i in range(len(w)):
        if w[i]==1:
            x.append(i+1) # positions run from 1 to L, not 0 to L-1
    return x

def fclosed(w,k):
    x=positions(w)
    z=0
    for t in mylist(len(k)):
        y=kperm(k,t)
        v=0
        for i in range(len(k)):
            v=v+y[i]*x[i]
        v=np.exp(1.j*v)
        z=z+e(t)*Aclosed(y)*v
    return z

## The function $f(w,k)$ for the *open* XXZ chain

The Hamiltonian for the open spin-1/2 XXZ chain, which depends on three parameters $\Delta$, $h$, $h'$, is given by

$$\mathcal{H} = \tfrac{1}{4}\sum_{n=1}^{L-1}
    \left(\sigma^x_n \sigma^x_{n+1} + \sigma^y_n \sigma^y_{n+1} + \Delta\,  \sigma^z_n \sigma^z_{n+1} \right) + \tfrac{1}{4}\left(h\, \sigma^z_1 + h'\, \sigma^z_L \right) $$


Here we make the open-chain function $f(w, \vec{k})$ for a given set of Bethe roots $\vec k=[k_1,\dots,k_M]$

$$f(w, \vec{k}) = \sum_{\sigma\in S_M}
    \sum_{\epsilon_1,\dots,\epsilon_M=\pm 1}
    \varepsilon(\sigma)\,\epsilon_1\dots\epsilon_M\, A(\epsilon_1 k_{\sigma(1)}, \ldots, \epsilon_M k_{\sigma(M)})\, 
    e^{i \sum_{j=1}^M \epsilon_j k_{\sigma(j)} x_j} $$

where $A(k_1, \ldots, k_M)$ is defined in Appendix A.

Only a few new things need to be defined:

In [4]:
# mysign gives all possible choices of [e0,e1,...,eM-1], where e=\pm 1
def mysign(M):
    return list(itertools.product([-1,1],repeat=M))

def B(k,kp):
    return s(k,kp)*s(kp,-k)

def beta(k):
    return (1+(hp-Delta)*np.exp(-1j*k))*np.exp(1j*(L+1)*k)

# k=[k0,...,kM-1]
def Aopen(k,sign):
    x=1
    for l in range(len(k)):
        x=x*beta(-sign[l]*k[l])
    for l in range(len(k)):
        for j in range(l):
            x=x*B(-sign[j]*k[j],sign[l]*k[l])*np.exp(-1j*sign[l]*k[l])
    return x

def fopen(w,k):
    x=positions(w)
    z=0
    for sign in mysign(len(k)): # all negations of bethe roots
        numnegatives=0
        for i in sign:
            if i==-1:
                numnegatives=numnegatives+1
        for t in mylist(len(k)): # all permutations of bethe roots
            y=kperm(k,t)
            v=0
            for i in range(len(k)):
                v=v+sign[i]*y[i]*x[i]
            v=np.exp(1.j*v)
            z=z+e(t)*((-1)**numnegatives)*Aopen(y,sign)*v
    return z

## Bethe circuit

In [5]:
# Create set of permutations
def P(L,M):
    if M==0:
        return [[0] * L]
    elif L-M==0:
        return [[1] * L]
    else:
        x=[]
        for p0 in P(L-1,M):
            x.append(p0+[0])
        for p1 in P(L-1,M-1):
            x.append(p1+[1])    
        return x

# here farray is a list of the evaluated coefficients f(w,k)
def F(b):
    sol=[] # solutions of w=ab
    for w in P(L,M):
        if w[L-len(b):L]==b:
            sol.append(w)
    if len(sol)==1:
        position=P(L,M).index(sol[0])
        return farray[position][1]
    else:  
        x=0
        for p in sol:
            position=P(L,M).index(p)
            x=x+abs(farray[position][1])**2
        return x**0.5

def G(i,b):
    x=[i] + b
    return F(x)/F(b)

def theta(b):
    return 2*np.arccos(abs(G(1,b)))
def lam(b):
    return np.angle(G(0,b))-np.pi
def phi(b):
    return np.angle(G(1,b))-lam(b)

def u(b):
    t=theta(b)
    la=lam(b)
    p=phi(b)
    return UGate(t,p,la)

def applyX(bethe,m,l):
    bethe.cx(L-m,L-m+l)

def applyI(bethe,m,l):
    sol=[] # these will be the strings b in P(L-m,M-l)
    for b in P(L-m,M-l):
        if F(b)!=0: # Check that F(b)!=0
            sol.append(b)
    if len(sol)==1:
        applyX(bethe,m,l)
        b=sol[0] # when b is unique, no extra controls are needed
        if l==1:
            cU = u(b).control(1)
            bethe.append(cU,[L-m+l,L-m])
        else:
            ccU = u(b).control(2)
            bethe.append(ccU,[L-m+l-1,L-m+l,L-m])
        applyX(bethe,m,l)
    elif len(sol)>1:
        applyX(bethe,m,l)
        for b in sol:
            extra=[] # these are going to be the locations of the extra controls
            for i in range(len(b)):
                if b[i]==1:
                    # if b=[0,1] then extra=[0], location of control
                    extra.append(len(b)-i-1)
            if l==1:
                cU = u(b).control(1+len(extra))
                bethe.append(cU,extra + [L-m+l,L-m])
            else:
                ccU = u(b).control(2+len(extra))
                bethe.append(ccU,extra + [L-m+l-1,L-m+l,L-m])
        applyX(bethe,m,l)   
        
def circuit(params,chain):
    L=params[0]
    M=params[1]
    Delta=params[2]
    k=params[3]
    if chain=='open':
        h=params[4]
        hp=params[5]
        
    q=QuantumRegister(L)
    bethe = QuantumCircuit(q)

    
    # Create initial state        
    for i in range(M):
        bethe.x(i) 
    bethe.barrier()

    # Apply I-gates
    for m in reversed(range(2,L+1)):
        for l in range(max(M+m-L,1),min(M,m-1)+1):
            applyI(bethe,m,l)       
        bethe.barrier() 
    return bethe

## Example Bethe root choices

For a given model (spin-1/2 XXZ closed or open), and for given values of $L$ and $M$, the Bethe roots $\{k_1, ..., k_M\}$ are assumed to be known. (The Bethe roots can be obtained by solving the corresponding Bethe equations, e.g. using Newton's method.) Below are some example choices:

Closed XXX, delta=1, Bethe roots={-2.0943951115532506, 2.094395111553251}, energy=6, L=4, M=2

---
Closed XXX, delta=1, Bethe roots={-1.7227684517685768, 3.141592653589793, 1.7227684517685768}, energy=8.60555128297041, L=6, M=3
    
---

Closed XXX, delta=1, Bethe roots={-1.5220024980135645, -2.634831769525861, 2.6348317695258605, 1.5220024980135642}, energy=11.302186879339416, L=8, M=4

---
Closed XXX, delta=1, Bethe roots={-1.3926185292093711, -2.3253446892906506, 3.141592653589793, 2.325344689290651, 1.3926185292093705}, energy=14.030892676785452, L=10, M=5

---
Closed XXZ, delta=0.5, Bethe roots={-2.2056631988105195, 2.2056631988105195}, energy=4.3722813261097535, L=4, M=2


---
Closed XXZ, delta=0.5, Bethe roots   {-1.884955585326176, 1.884955598981455, 3.1415926606099034}, energy=6.23606797749956, L=6, M=3

---
Closed XXZ, delta=Cosh(0.1)=1.0050041680558035, Bethe roots={0.011213836970837356, 1.041590632711175 - 0.729100186957427j, 1.041590632711175 + 0.729100186957427j}, energy=1.4498157711985113, L=6, M=3

---
Closed XXZ, delta=0.5, Bethe roots={-2.6839470901091143, -1.717726341337345, 1.717726341337345, 2.6839470901091143}, energy=8.173988723280036, L=8, M=4

---
Open XXX, delta=1, h=hp=0, Bethe roots={2.276165741027847, 1.2846162678803887}, energy=4.732050845799164, L=4, M=2

---
Open XXX, delta=1, h=hp=0, Bethe roots={2.5398342001187024, 1.9084069140934237, 1.1461507768412273}, energy=7.487154232133537, L=6, M=3

---
Open XXX, delta=1, h=hp=0, Bethe roots={2.679744980313025, 2.2056671514397728, 1.6965261329177208, 1.0601164701419314}, energy=10.249865096564593, L=8, M=4


---
Open XXZ, delta=0.5, h=0.1, hp=0.3, Bethe roots={0.682741231585585, 1.385611917077896}, energy=0.08005214893738843, L=4, M=2

---
Open XXZ, delta=0.5, h=0.1, hp=0.3, Bethe roots={0.3691975274494157, 0.7130354857997001, 0.9909893600637687}, energy=-1.473718892682273, L=6, M=3


## Running the circuit - example 1: open chain 

In [6]:
# choose the desired values
chain='open'
L=4
M=2
Delta=0.5
k=[0.682741231585585, 1.385611917077896]
h=0.1
hp=0.3


# store the coefficients f(w,k) and run the Bethe circuit
farray=[]
if chain=='closed':
    for w in P(L,M):
        farray=farray+[[w,fclosed(w,k)]]
    bethe=circuit([L,M,Delta,k],'closed')
    
if chain=='open':
    for w in P(L,M):
        farray=farray+[[w,fopen(w,k)]]
    bethe=circuit([L,M,Delta,k,h,hp],'open')

bethe.draw(output='mpl', initial_state=True,scale=1,plot_barriers=True,fold=40)

### We build the Hamiltonians out of SparsePauliOps

In [7]:
from qiskit.quantum_info import SparsePauliOp

op=[]
for i in range(L-1):
    op=op+[("XX",[i,i+1],-1/2)]
    op=op+[("YY",[i,i+1],-1/2)]
    op=op+[("ZZ",[i,i+1],-Delta/2)]

if chain=='closed':
    op=op+[("XX",[L-1,0,-1/2],-1/2)]
    op=op+[("YY",[L-1,0,-1/2],-1/2)]
    op=op+[("ZZ",[L-1,0,-1/2],-Delta/2)]
    op=op+[("I",[0],Delta*L/2)]
    
if chain=='open':
    #recall that qiskit labels the vector spaces from left to right
    op=op+[("Z",[L-1],-h/2)]
    op=op+[("Z",[0],-hp/2)]
    op=op+[("I",[0],(h+hp+Delta*(L-1))/2)]

ham = SparsePauliOp.from_sparse_list(op, num_qubits=L)

### Compute the expectation value of the state using shots, and compare it with the energy from the Bethe ansatz:

In [8]:
from qiskit_aer.primitives import Estimator as AerEstimator

aer_estimator = AerEstimator(run_options={"shots": 10000})

def evaluate_energy(qc,ham):
    #qc.assign_parameters({new: x0})
    estimator_job1 = aer_estimator.run(qc, ham)
    estimator_result1  = estimator_job1.result()
    return estimator_result1.values[0]

In [9]:
evaluate_energy(bethe,ham)

0.08084000000000002

In [10]:
# energy from Bethe ansatz
exact_energy=0
for i in k:
    exact_energy=exact_energy+2*(Delta-np.cos(i))
print(exact_energy)

0.08005214893738843


### Output the $2^L$ vector using the statevector simulator

In [11]:
from qiskit.quantum_info import Statevector

sv = Statevector.from_label('0'*L)
ft_sv = sv.evolve(bethe)
ft_sv.data

array([ 0.        +0.j,  0.        +0.j,  0.        +0.j, -0.56010015+0.j,
        0.        +0.j, -0.4032429 +0.j, -0.07864458-0.j,  0.        +0.j,
        0.        +0.j, -0.05481203-0.j,  0.33089399+0.j,  0.        +0.j,
        0.63639832+0.j,  0.        +0.j,  0.        +0.j,  0.        +0.j])

In [12]:
ft_sv.draw('latex')

<IPython.core.display.Latex object>

### Explicit check of the eigenvector and the corresponding eigenvalue

In [13]:
np.allclose((ham.to_matrix())@(ft_sv.data),exact_energy*(ft_sv.data),atol=1e-3)

True

## Running the circuit - example 2: closed chain 

In [15]:
# choose the desired values
chain='closed'
L=6
M=3
Delta=1.0050041680558035
k=[0.011213836970837356, 1.041590632711175 - 0.729100186957427j, 1.041590632711175 + 0.729100186957427j]
#h=.1
#hp=0.3


# store the coefficients f(w,k) and run the Bethe circuit
farray=[]
if chain=='closed':
    for w in P(L,M):
        farray=farray+[[w,fclosed(w,k)]]
    bethe=circuit([L,M,Delta,k],'closed')
    
if chain=='open':
    for w in P(L,M):
        farray=farray+[[w,fopen(w,k)]]
    bethe=circuit([L,M,Delta,k,h,hp],'open')

bethe.draw(output='mpl', initial_state=True,scale=1,plot_barriers=True,fold=40)

In [16]:
op=[]
for i in range(L-1):
    op=op+[("XX",[i,i+1],-1/2)]
    op=op+[("YY",[i,i+1],-1/2)]
    op=op+[("ZZ",[i,i+1],-Delta/2)]

if chain=='closed':
    op=op+[("XX",[L-1,0,-1/2],-1/2)]
    op=op+[("YY",[L-1,0,-1/2],-1/2)]
    op=op+[("ZZ",[L-1,0,-1/2],-Delta/2)]
    op=op+[("I",[0],Delta*L/2)]
    
if chain=='open':
    op=op+[("Z",[0],-h/2)]
    op=op+[("Z",[L-1],-hp/2)]
    op=op+[("I",[0],(h+hp+Delta*(L-1))/2)]

ham = SparsePauliOp.from_sparse_list(op, num_qubits=L)

In [17]:
evaluate_energy(bethe,ham)

1.4454014964983426

In [18]:
# energy from Bethe ansatz
exact_energy=0
for i in k:
    exact_energy=exact_energy+2*(Delta-np.cos(i))
print(exact_energy)

(1.4498157711985125+0j)


In [19]:
sv = Statevector.from_label('0'*L)
ft_sv = sv.evolve(bethe)
ft_sv.data

array([ 0.        +0.j        ,  0.        -0.j        ,
       -0.        -0.j        ,  0.        +0.j        ,
       -0.        -0.j        ,  0.        +0.j        ,
        0.        +0.j        ,  0.37955524-0.j        ,
       -0.        +0.j        ,  0.        +0.j        ,
        0.        +0.j        ,  0.10631201-0.j        ,
        0.        +0.j        , -0.05315601-0.0920689j ,
       -0.18977762-0.32870448j,  0.        +0.j        ,
        0.        -0.j        ,  0.        +0.j        ,
        0.        +0.j        , -0.05315601+0.0920689j ,
        0.        +0.j        , -0.        -0.j        ,
       -0.05315601-0.0920689j ,  0.        +0.j        ,
        0.        +0.j        ,  0.10631201-0.j        ,
       -0.05315601+0.0920689j ,  0.        +0.j        ,
       -0.18977762+0.32870448j,  0.        +0.j        ,
        0.        +0.j        , -0.        +0.j        ,
       -0.        +0.j        ,  0.        +0.j        ,
        0.        +0.j        ,

In [20]:
ft_sv.draw('latex')

<IPython.core.display.Latex object>

In [23]:
np.allclose((ham.to_matrix())@(ft_sv.data),exact_energy*(ft_sv.data),atol=1e-6)

True